In [1]:
import sys
sys.path.append("../")

In [2]:
from src.utils.cache import get_cache_client
cache =  get_cache_client()

In [12]:
from src.utils.cache import get_cache_client
r = get_cache_client()
try:
    total = len(await r.keys("*"))
    print(total)
finally:
    await r.close()
    await r.connection_pool.disconnect()

2


/tmp/ipykernel_300/3560370905.py:7: DeprecationWarning: Call to deprecated close. (Use aclose() instead) -- Deprecated since version 5.0.1.
  await r.close()


In [13]:
# Async (e.g., notebook or running event loop)
from src.utils.cache import get_cache_client

r = get_cache_client()
try:
    keys = []
    # use SCAN to avoid blocking; count can be tuned
    async for k in r.scan_iter(match="*", count=100):
        keys.append(k)
        if len(keys) >= 10:
            break
    print(keys)
finally:
    await r.close()
    await r.connection_pool.disconnect()

['query:string:7641409547018718388', 'query:string:1340722539853643331']


/tmp/ipykernel_300/1238699391.py:14: DeprecationWarning: Call to deprecated close. (Use aclose() instead) -- Deprecated since version 5.0.1.
  await r.close()


In [14]:
# in a notebook cell
try:
    print(await r.get("some:key"))
finally:
    await r.close()
    await r.connection_pool.disconnect()


None


/tmp/ipykernel_300/1785143273.py:5: DeprecationWarning: Call to deprecated close. (Use aclose() instead) -- Deprecated since version 5.0.1.
  await r.close()


In [15]:
from src.utils.cache import get_cache_client
import json

r = get_cache_client()
try:
    for k in ['query:string:7641409547018718388', 'query:string:1340722539853643331']:
        t = await r.type(k)
        if t == 'string':
            raw = await r.get(k)
            try:
                val = json.loads(raw) if raw is not None else None
            except Exception:
                val = raw
        elif t == 'hash':
            val = await r.hgetall(k)
        elif t == 'list':
            val = await r.lrange(k, 0, -1)
        elif t == 'set':
            val = await r.smembers(k)
        elif t == 'zset':
            val = await r.zrange(k, 0, -1, withscores=True)
        else:
            val = f"<unknown type {t}>"
        print(k, "->", t, "->", val)
finally:
    await r.close()
    await r.connection_pool.disconnect()


query:string:7641409547018718388 -> string -> {'query_id': 'query_7641409547018718388', 'answer': 'Mock response to: \n        You are a research assistant helping a re... This is a simulated LLM response for development purposes.', 'citations': [], 'confidence': 0.8, 'suggested_follow_up_questions': ['What else would you like to explore?', 'Any clarifying details needed?', 'Related experiments to consider?'], 'processed_at': '2025-09-28T11:29:49.311721'}
query:string:1340722539853643331 -> string -> {'query_id': 'query_1340722539853643331', 'answer': 'Mock response to: \n        You are a research assistant helping a re... This is a simulated LLM response for development purposes.', 'citations': [], 'confidence': 0.8, 'suggested_follow_up_questions': ['What else would you like to explore?', 'Any clarifying details needed?', 'Related experiments to consider?'], 'processed_at': '2025-09-28T11:31:13.299495'}


/tmp/ipykernel_300/426261519.py:26: DeprecationWarning: Call to deprecated close. (Use aclose() instead) -- Deprecated since version 5.0.1.
  await r.close()
